In [1]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv('CovidToy.csv')

In [5]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'], test_size=0.2)

In [9]:
X_train

,age,gender,fever,cough,city
96,51,Female,101.0,Strong,Kolkata
63,10,Male,100.0,Mild,Bangalore
84,69,Female,98.0,Strong,Mumbai
64,42,Male,104.0,Mild,Mumbai
56,71,Male,NaN,Strong,Kolkata
...,...,...,...,...,...
3,31,Female,98.0,Mild,Kolkata
20,12,Male,98.0,Strong,Bangalore
9,64,Female,101.0,Mild,Delhi
16,69,Female,103.0,Mild,Kolkata


### Hactic

In [14]:
# Adding simple imputer to fever col.
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# Also the test data.
X_test_fever = si.fit_transform(X_test[['fever']])                  
X_train_fever.shape

(80, 1)

In [16]:
# Ordinalencoding to Cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# Also the test data.
X_test_cough = oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [19]:
# OneHotEncoding to Gender, City
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# Also the test data.
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])
X_train_gender_city.shape

(80, 4)

In [20]:
# Extracting Age.
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# Also the test data.
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values
X_train_age.shape

(80, 1)

In [21]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

# Also the test data.
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)
X_train_transformed.shape

(80, 7)

### Fast

In [22]:
from sklearn.compose import ColumnTransformer

In [23]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
], remainder='passthrough')

In [24]:
transformer.fit_transform(X_train).shape

(80, 7)

In [25]:
transformer.fit_transform(X_test).shape

(20, 7)

In [31]:
transformer.fit_transform(X_train)

array([[101. ,   1. ,   0. ,   0. ,   1. ,   0. ,  51. ],
       [100. ,   0. ,   1. ,   0. ,   0. ,   0. ,  10. ],
       [ 98. ,   1. ,   0. ,   0. ,   0. ,   1. ,  69. ],
       [104. ,   0. ,   1. ,   0. ,   0. ,   1. ,  42. ],
       [100.8,   1. ,   1. ,   0. ,   1. ,   0. ,  71. ],
       [ 99. ,   0. ,   0. ,   0. ,   0. ,   1. ,  60. ],
       [ 98. ,   0. ,   1. ,   0. ,   1. ,   0. ,  24. ],
       [104. ,   1. ,   0. ,   0. ,   1. ,   0. ,  54. ],
       [104. ,   0. ,   0. ,   0. ,   0. ,   0. ,  18. ],
       [ 98. ,   0. ,   0. ,   0. ,   0. ,   1. ,  65. ],
       [104. ,   1. ,   0. ,   1. ,   0. ,   0. ,  34. ],
       [ 98. ,   0. ,   0. ,   0. ,   1. ,   0. ,  26. ],
       [100.8,   1. ,   0. ,   0. ,   0. ,   0. ,  42. ],
       [103. ,   0. ,   0. ,   1. ,   0. ,   0. ,  73. ],
       [ 99. ,   0. ,   0. ,   0. ,   0. ,   0. ,  22. ],
       [104. ,   1. ,   0. ,   1. ,   0. ,   0. ,  75. ],
       [101. ,   1. ,   0. ,   1. ,   0. ,   0. ,  68. ],
       [104. ,

In [30]:
transformer.fit_transform(X_test)

array([[100.,   0.,   1.,   1.,   0.,   0.,  27.],
       [104.,   0.,   0.,   0.,   0.,   0.,  12.],
       [ 98.,   1.,   0.,   0.,   0.,   1.,  81.],
       [102.,   0.,   1.,   0.,   0.,   1.,  74.],
       [102.,   1.,   1.,   1.,   0.,   0.,  20.],
       [ 98.,   1.,   0.,   1.,   0.,   0.,  40.],
       [102.,   1.,   0.,   1.,   0.,   0.,  33.],
       [ 99.,   1.,   0.,   0.,   1.,   0.,  25.],
       [103.,   0.,   1.,   0.,   1.,   0.,  83.],
       [ 98.,   1.,   1.,   0.,   0.,   1.,  23.],
       [102.,   1.,   0.,   0.,   1.,   0.,  82.],
       [103.,   0.,   1.,   0.,   1.,   0.,  60.],
       [101.,   0.,   0.,   1.,   0.,   0.,  49.],
       [101.,   0.,   0.,   0.,   0.,   1.,  65.],
       [101.,   0.,   0.,   0.,   1.,   0.,   8.],
       [103.,   1.,   1.,   0.,   0.,   0.,  46.],
       [ 99.,   0.,   1.,   0.,   0.,   0.,  72.],
       [102.,   0.,   1.,   0.,   1.,   0.,   5.],
       [ 98.,   0.,   1.,   1.,   0.,   0.,  83.],
       [104.,   0.,   0.,   0.,